<a href="https://colab.research.google.com/github/gcallj/test/blob/main/TCC_Ensemble_learning_mix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# ───────────────────────────────────────────────────────────────
!pip install -q \
    scikit-learn scikit-optimize \
    tensorflow scikeras\
    lightgbm tqdm joblib seaborn pyarrow typing-extensions xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 3.6 MB/s eta 0:00:00


In [3]:
# ═══════════════ USER OPTIONS ══════════════════════════════════
#EXCLUDE_BATTERIES = ['B0049', 'B0050']   # baterias para excluir
EXCLUDE_BATTERIES = []                 # para usar todos

# Forçar retreinamento em todos os modelos
LOAD_LGBM = True
LOAD_RF   = True
LOAD_DNN  = True
LOAD_SVR  = True  # modelo SVR
LOAD_KNN  = True  # Gaussian Process Regressor
LOAD_META = True
LOAD_XGB = True
LOAD_MLPRES = True

MODEL_DIR = '/content/gdrive/My Drive/Colab Notebooks/Batteries_TCC/models/v2'
DATA_FILE = 'gdrive/My Drive/Colab Notebooks/Batteries_TCC/battery_files_aggregated_v2.parquet'

In [4]:
# ─── Imports ─────────────────────────────────────────────────────
from google.colab import drive
import os, warnings, inspect, joblib
import numpy as np
import pandas as pd
import lightgbm as lgb
import tensorflow as tf
from tqdm.auto import tqdm
from joblib import parallel_backend
from sklearn.model_selection import GroupShuffleSplit, GroupKFold, LeaveOneGroupOut
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from skopt import BayesSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C, WhiteKernel
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import RidgeCV
from sklearn.base import BaseEstimator, clone
from scikeras.wrappers import KerasRegressor, BaseWrapper

BaseWrapper.__sklearn_tags__ = BaseEstimator.__sklearn_tags__
warnings.filterwarnings("ignore")
SEED = 42
np.random.seed(SEED)
tf.keras.utils.set_random_seed(SEED)

drive.mount('/content/gdrive')
os.makedirs(MODEL_DIR, exist_ok=True)


Mounted at /content/gdrive


# 0 - Helpers

In [5]:
# ─── Helpers ─────────────────────────────────────────────────────
def save(obj, name):
    joblib.dump(obj, os.path.join(MODEL_DIR, name))

def load(name):
    path = os.path.join(MODEL_DIR, name)
    return joblib.load(path) if os.path.exists(path) else None

def bayes_search_fit(opt, bar, X, y, groups=None):
    done = 0
    def _cb(_):
        nonlocal done; bar.update(); done += 1
    with parallel_backend('threading'):
        opt.fit(X, y, groups=groups, callback=_cb)
    bar.update(bar.total - done); bar.close()

def metrics(tag, y_true_pct, y_pred_pct, cyc_max):
    y_true = y_true_pct * cyc_max
    y_pred = y_pred_pct * cyc_max
    r2 = r2_score(y_true_pct, y_pred_pct)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    print(f"{tag:<12} R²(pct)={r2:.4f}  RMSE(cic)={rmse:.1f}  MAE(cic)={mae:.1f}")

def per_batt(tag, batt, y_t, y_p):
    print(f"{tag} – Por Bateria R²")
    vals=[r2_score(y_t[batt==b], y_p[batt==b]) for b in sorted(batt.unique())]
    for b,v in zip(sorted(batt.unique()), vals):
        print(f"  {b}: {v:.4f}")
    print("  Media:", np.mean(vals).round(4), '\n')

# ─── Load & Preprocess Data ─────────────────────────────────────
_df = pd.read_parquet(DATA_FILE)
_df = _df[~_df.battery_id.isin(EXCLUDE_BATTERIES)].copy()
_df['meta_capacity'] = pd.to_numeric(_df['meta_capacity'], errors='coerce')

def rm_outliers(g):
    q1, q3 = g.meta_capacity.quantile([.25, .75])
    iqr = q3 - q1
    return g[g.meta_capacity.between(q1-1.5*iqr, q3+1.5*iqr)]

_df = _df.groupby('battery_id', group_keys=False).apply(rm_outliers)
_df['RUL_cycles'] = _df.groupby('battery_id')['cycle'].transform('max') - _df['cycle']
_df['RUL_pct']    = _df['RUL_cycles'] / _df.groupby('battery_id')['cycle'].transform('max')
_df['rated_capacity']   = _df.sort_values('cycle').groupby('battery_id')['meta_capacity'].transform('first')
_df['percent_capacity'] = _df['meta_capacity'] / _df['rated_capacity']
cutoff = {**{f"B00{i:02d}":thr for i,thr in zip(range(5,19),[2.7,2.5,2.2,2.5,2.0,2.2,2.5,2.7,2.0,2.2,2.5,2.7,2.0,2.2])},
          **{f"B00{i:02d}":v for i,v in zip(range(25,57),[2.0,2.2,2.5,2.7]*8)}}
_df['discharge_cutoff'] = _df['battery_id'].map(cutoff)
_df['voltage_margin']   = _df['min_voltage'] - _df['discharge_cutoff']

FEATURES = [
    'ambient_temperature','meta_capacity','duration','capacity_mAh','energy_Wh',
    'min_voltage','max_voltage','voltage_range','avg_current','min_current','max_current',
    'current_range','avg_temperature','min_temperature','max_temperature','temperature_range',
    'energy_to_capacity','percent_capacity','voltage_margin','discharge_cutoff','cycle',
    'impedance_real','impedance_imag','impedance_magnitude','impedance_phase_deg',
    'charge_capacity_mAh','charge_energy_Wh','charge_min_voltage','charge_max_voltage','charge_voltage_range',
    'charge_avg_current','charge_min_current','charge_max_current','charge_current_range',
    'charge_avg_temperature','charge_min_temperature','charge_max_temperature','charge_temperature_range',
    'charge_energy_to_capacity'
]

X = _df[FEATURES]
y_pct = _df['RUL_pct']
groups = _df['battery_id']

# Splits
TR, TE = next(GroupShuffleSplit(test_size=0.3, random_state=SEED).split(X, y_pct, groups))
X_tr, X_te = X.iloc[TR], X.iloc[TE]
y_tr, y_te = y_pct.iloc[TR], y_pct.iloc[TE]
g_tr, g_te = groups.iloc[TR], groups.iloc[TE]
CYC_MAX_TE = _df.groupby('battery_id')['cycle'].transform('max').iloc[TE].values
cv_lobo = GroupKFold(n_splits=g_tr.nunique())
logo    = LeaveOneGroupOut()

print("🎯 Baterias treino:", sorted(g_tr.unique()))
print("🧪 Baterias teste:", sorted(g_te.unique()), "\n")

🎯 Baterias treino: ['B0006', 'B0007', 'B0018', 'B0025', 'B0026', 'B0027', 'B0028', 'B0031', 'B0032', 'B0034', 'B0036', 'B0039', 'B0040', 'B0041', 'B0043', 'B0045', 'B0046', 'B0048', 'B0051', 'B0052', 'B0053', 'B0054', 'B0056']
🧪 Baterias teste: ['B0005', 'B0029', 'B0030', 'B0033', 'B0038', 'B0042', 'B0044', 'B0047', 'B0049', 'B0050', 'B0055'] 



# 1 - LightGBM

In [6]:
# ─── LightGBM ─────────────────────────────────────────────────────
m_lgb = load('lgbm.pkl') if LOAD_LGBM else None
if m_lgb is None:
    print("► Treinando LGBM...")
    space = {
        'num_leaves': (20,120), 'max_depth': (3,25),
        'learning_rate': (1e-3,0.3,'log-uniform'),
        'n_estimators': (50,300), 'subsample': (0.5,1.0), 'colsample_bytree': (0.5,1.0)
    }
    opt = BayesSearchCV(lgb.LGBMRegressor(random_state=SEED, verbose=-1, verbosity=-1), space,
                        n_iter=30, cv=cv_lobo, scoring='r2', n_jobs=-1, random_state=SEED)
    bar = tqdm(total=30, desc='LGBM')
    bayes_search_fit(opt, bar, X_tr, y_tr, groups=g_tr)
    m_lgb = opt.best_estimator_
    save(m_lgb, 'lgbm.pkl')
m_lgb.set_params(verbose=-1, verbosity=-1)
print(">> LGBM params:", m_lgb.get_params())
pred_lgb = m_lgb.predict(X_te)
metrics("LGBM", y_te, pred_lgb, CYC_MAX_TE)
per_batt("LGBM", g_te, y_te, pred_lgb)



>> LGBM params: {'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 0.768657153297323, 'importance_type': 'split', 'learning_rate': 0.08801489600316921, 'max_depth': 17, 'min_child_samples': 20, 'min_child_weight': 0.001, 'min_split_gain': 0.0, 'n_estimators': 177, 'n_jobs': None, 'num_leaves': 97, 'objective': None, 'random_state': 42, 'reg_alpha': 0.0, 'reg_lambda': 0.0, 'subsample': 0.9020270650614294, 'subsample_for_bin': 200000, 'subsample_freq': 0, 'verbose': -1, 'verbosity': -1}
LGBM         R²(pct)=0.9043  RMSE(cic)=11.3  MAE(cic)=7.3
LGBM – Por Bateria R²
  B0005: 0.9891
  B0029: 0.9808
  B0030: 0.9805
  B0033: 0.8614
  B0038: 0.9571
  B0042: 0.8356
  B0044: 0.9719
  B0047: 0.9534
  B0049: 0.2017
  B0050: 0.4840
  B0055: 0.9650
  Media: 0.8346 



# 2 - Random Forest

In [7]:
# ─── Random Forest ────────────────────────────────────────────────
m_rf = load('rf.pkl') if LOAD_RF else None
if m_rf is None:
    print("► Treinando RF...")
    space = {
        'n_estimators': (100,600), 'max_depth': (3,10),
        'min_samples_split': (2,25), 'min_samples_leaf': (2,20),
        'max_features': ['sqrt'], 'bootstrap': [False]
    }
    opt = BayesSearchCV(RandomForestRegressor(random_state=SEED, n_jobs=-1), space,
                        n_iter=30, cv=cv_lobo, scoring='r2', random_state=SEED)
    bar = tqdm(total=30, desc='RF')
    bayes_search_fit(opt, bar, X_tr, y_tr, groups=g_tr)
    m_rf = opt.best_estimator_
    save(m_rf, 'rf.pkl')
print(">> RF params:", m_rf.get_params())
pred_rf = m_rf.predict(X_te)
metrics("RF", y_te, pred_rf, CYC_MAX_TE)
per_batt("RF", g_te, y_te, pred_rf)

>> RF params: {'bootstrap': False, 'ccp_alpha': 0.0, 'criterion': 'squared_error', 'max_depth': 9, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 2, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'monotonic_cst': None, 'n_estimators': 177, 'n_jobs': -1, 'oob_score': False, 'random_state': 42, 'verbose': 0, 'warm_start': False}
RF           R²(pct)=0.7751  RMSE(cic)=17.5  MAE(cic)=12.2
RF – Por Bateria R²
  B0005: 0.9833
  B0029: 0.8643
  B0030: 0.7865
  B0033: 0.6861
  B0038: 0.6928
  B0042: 0.7060
  B0044: 0.8433
  B0047: 0.7833
  B0049: 0.2347
  B0050: 0.5232
  B0055: 0.7501
  Media: 0.714 



# 3 - SVR

In [8]:
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import RobustScaler
from skopt import BayesSearchCV
from skopt.space import Real, Categorical
from sklearn.model_selection import GroupKFold

# Selecione as top-30 features mais importantes do LGBM
top_features = list(pd.Series(m_lgb.feature_importances_, index=FEATURES).nlargest(7).index)
X_tr_top = X_tr[top_features]
X_te_top = X_te[top_features]

svr_pipe = Pipeline([
    ('imp', SimpleImputer(strategy='median')),
    ('scl', RobustScaler()),
    ('svr', SVR())
])

param_space = {
    'svr__C': Real(1e-2, 1e2, prior='log-uniform'),
    'svr__epsilon': Real(1e-4, 0.2, prior='log-uniform'),
    'svr__gamma': Real(1e-4, 1.0, prior='log-uniform'),
    'svr__kernel': Categorical(['rbf'])
}

cv_svr = GroupKFold(n_splits=5)
opt = BayesSearchCV(
    svr_pipe,
    param_space,
    n_iter=12,          # tuning rápido, pode aumentar
    cv=cv_svr,
    n_jobs=-1,
    scoring='r2',
    random_state=SEED
)
bar = tqdm(total=opt.n_iter, desc="SVR-topF")
bayes_search_fit(opt, bar, X_tr_top, y_tr, groups=groups.iloc[TR])
m_svr = opt.best_estimator_
save(m_svr, "svr_topf.pkl")

print("\nMelhores parâmetros SVR (top features):")
for k, v in opt.best_params_.items():
    print(f"  {k}: {v}")

pred_svr = m_svr.predict(X_te_top)
metrics("SVR (top features)", y_te, pred_svr, CYC_MAX_TE)
per_batt("SVR (top features)", groups.iloc[TE], y_te, pred_svr)


SVR-topF:   0%|          | 0/12 [00:00<?, ?it/s]


Melhores parâmetros SVR (top features):
  svr__C: 36.939174331350394
  svr__epsilon: 0.02110922772654
  svr__gamma: 0.33098662965053316
  svr__kernel: rbf
SVR (top features) R²(pct)=0.7520  RMSE(cic)=17.8  MAE(cic)=11.9
SVR (top features) – Por Bateria R²
  B0005: 0.9889
  B0029: 0.9484
  B0030: 0.9660
  B0033: 0.6598
  B0038: 0.4188
  B0042: 0.9300
  B0044: 0.8643
  B0047: 0.9352
  B0049: -0.1601
  B0050: 0.1129
  B0055: 0.3779
  Media: 0.6402 



# 4 - KNN

In [9]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import Pipeline
from skopt import BayesSearchCV
from skopt.space import Integer, Categorical
from tqdm.auto import tqdm

knn_pkl = 'knn.pkl'
m_knn = load(knn_pkl) if 'LOAD_KNN' in globals() and LOAD_KNN else None
if m_knn is None:
    print("► Treinando KNeighborsRegressor (feature selection + tuning)...")
    knn_pipe = Pipeline([
        ('imp', SimpleImputer(strategy='median')),
        ('scl', StandardScaler()),
        ('sel', SelectKBest(score_func=f_regression, k=15)),
        ('knn', KNeighborsRegressor())
    ])
    space = {
        'sel__k': Integer(8, 25),
        'knn__n_neighbors': Integer(2, 30),
        'knn__weights': Categorical(['uniform', 'distance']),
        'knn__p': Integer(1, 2)
    }
    opt = BayesSearchCV(
        knn_pipe, space,
        n_iter=30, cv=5, scoring='r2', n_jobs=-1, random_state=SEED
    )
    bar = tqdm(total=30, desc='KNN')
    bayes_search_fit(opt, bar, X_tr, y_tr, groups=groups.iloc[TR])
    m_knn = opt.best_estimator_
    save(m_knn, knn_pkl)
    print("\nKNN best params:")
    for k, v in opt.best_params_.items():
        print(f"  {k}: {v}")
else:
    print("► KNN loaded")
    print("\nKNN pipeline steps & params:")
    print(m_knn.named_steps)

pred_knn = m_knn.predict(X_te)
metrics("KNN", y_te, pred_knn, CYC_MAX_TE)
per_batt("KNN", groups.iloc[TE], y_te, pred_knn)


► KNN loaded

KNN pipeline steps & params:
{'imp': SimpleImputer(strategy='median'), 'scl': StandardScaler(), 'sel': SelectKBest(k=22, score_func=<function f_regression at 0x7c9ef5e34e00>), 'knn': KNeighborsRegressor(n_neighbors=17, weights='distance')}
KNN          R²(pct)=0.8470  RMSE(cic)=10.3  MAE(cic)=7.1
KNN – Por Bateria R²
  B0005: 0.9772
  B0029: 0.9319
  B0030: 0.5612
  B0033: 0.9223
  B0038: 0.1763
  B0042: 0.8926
  B0044: 0.9010
  B0047: 0.8624
  B0049: 0.4295
  B0050: 0.4979
  B0055: 0.9572
  Media: 0.7372 



# 5 - DNN

In [10]:
# ──────────────────────────────────────────────────────────────────────────
#  DNN ensemble (5 redes) — sem battery_id como feature
# ──────────────────────────────────────────────────────────────────────────
import os, joblib, numpy as np, tensorflow as tf, warnings
from tensorflow.keras import layers, callbacks, optimizers, losses
from sklearn.base import BaseEstimator
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GroupKFold
from skopt import BayesSearchCV
from skopt.space import Integer, Real, Categorical
from scikeras.wrappers import KerasRegressor
from tqdm.auto import tqdm

# ─── parâmetros globais
NUM_MEMBERS = 5
CV_FOLDS    = 3
EPOCHS_MAX  = 200
N_ITER_BAY  = 30
BATCH_SIZE  = 256
PATIENCE    = 10
TF_SEEDS    = [42, 99, 1234, 2025, 31415][:NUM_MEMBERS]

tf.config.optimizer.set_jit(True)      # ativa XLA

# ─── pesos  √(max/freq)  (sem usar battery_id na feature matrix)
counts = groups.iloc[TR].value_counts()
w_tr = np.sqrt(counts.max() / groups.iloc[TR].map(counts)).values.astype(float)

# ─── pré-processamento numérico
prep = ColumnTransformer([
    ("num", Pipeline([
        ("imp", SimpleImputer(strategy="median")),
        ("scl", StandardScaler())
    ]), FEATURES)
])

# ─── construtor do modelo
def build_nn(meta,
             units=320, num_layers=4, activation="gelu",
             dropout=0.15, bn=True, learning_rate=2e-3,
             optimizer="adamw"):
    tf.keras.backend.clear_session()
    inp = layers.Input(shape=(meta["n_features_in_"],))
    x   = inp
    for _ in range(num_layers):
        x = layers.Dense(units, activation=activation,
                         kernel_initializer="he_normal")(x)
        if bn:
            x = layers.BatchNormalization()(x)
        x = layers.Dropout(dropout)(x)
    out = layers.Dense(1, activation="sigmoid")(x)
    opt_cls = {"adam": optimizers.Adam,
               "adamw": optimizers.AdamW,
               "nadam": optimizers.Nadam}[optimizer]
    model = tf.keras.Model(inp, out)
    model.compile(opt_cls(learning_rate), loss="log_cosh")
    return model

# ─── espaço de busca
space = {
    "net__model__units":        Integer(192, 512),
    "net__model__num_layers":   Integer(3, 6),
    "net__model__activation":   Categorical(["relu", "gelu", "selu"]),
    "net__model__dropout":      Real(0.05, 0.3),
    "net__model__bn":           Categorical([True, False]),
    "net__model__learning_rate":Real(5e-4, 5e-3, prior="log-uniform"),
    "net__model__optimizer":    Categorical(["adamw", "nadam", "adam"])
}

# ─── caminhos de artefatos
prep_path   = os.path.join(MODEL_DIR, "dnn_prep.pkl")
model_paths = [os.path.join(MODEL_DIR, f"dnn_{i}.keras") for i in range(NUM_MEMBERS)]
member_models = []

# ─── tenta carregar
if LOAD_DNN and os.path.exists(prep_path) and all(os.path.exists(p) for p in model_paths):
    print("► DNN ensemble carregado do disco")
    prep = joblib.load(prep_path)
    member_models = [tf.keras.models.load_model(p) for p in model_paths]
else:
    print("► Treinando DNN ensemble…")

    class ProgCB(callbacks.Callback):
        def __init__(self, bar): super().__init__(); self.bar = bar
        def on_epoch_end(self, epoch, logs=None): self.bar.update()

    best_params = None
    for i, sd in enumerate(TF_SEEDS):
        print(f"   • membro {i+1}/{NUM_MEMBERS} (seed={sd})")

        kr = KerasRegressor(
            model=build_nn,
            epochs=EPOCHS_MAX,
            batch_size=BATCH_SIZE,
            verbose=0,
            random_state=sd,
            validation_split=0.1,
            callbacks=[
                callbacks.EarlyStopping(patience=PATIENCE,
                                        restore_best_weights=True),
                callbacks.ReduceLROnPlateau(patience=PATIENCE//2,
                                            factor=0.3, min_lr=1e-6)
            ]
        )
        pipe = Pipeline([("prep", prep), ("net", kr)])

        if i == 0:          # —— busca Bayes (30 it) com barra
            bar = tqdm(total=N_ITER_BAY, desc=f"Bayes seed {sd}", ncols=80)
            def _cb(_): bar.update()
            search = BayesSearchCV(
                pipe, space, n_iter=N_ITER_BAY,
                cv=GroupKFold(CV_FOLDS), scoring="r2",
                random_state=sd, n_jobs=1
            )
            bar = tqdm(total=N_ITER_BAY, desc=f"Bayes seed {sd}", ncols=80)
            def _cb(_): bar.update()

            def bayes_search_fit(opt, bar, X, y, groups=None):
                done = 0
                def _cb(_):
                    nonlocal done; bar.update(); done += 1
                with parallel_backend('threading'):
                    opt.fit(X, y, groups=groups, callback=_cb)
                bar.update(bar.total - done); bar.close()

            from joblib import parallel_backend
            bayes_search_fit(
                search, bar, X_tr, y_tr,
                groups=groups.iloc[TR]
            )
            pipe = search.best_estimator_
            best_params = search.best_params_
        else:               # —— membros 2-5
            pipe.set_params(**best_params)
            bar = tqdm(total=EPOCHS_MAX, desc=f"seed {sd}",
                       leave=False, ncols=80)
            pipe.set_params(net__callbacks=[ProgCB(bar)])
            pipe.fit(X_tr, y_tr, net__sample_weight=w_tr)
            bar.close()

        model_i = pipe.named_steps["net"].model_
        model_i.save(model_paths[i])
        member_models.append(model_i)

    joblib.dump(prep, prep_path)

# ─── Ensemble wrapper (auto-fit do prep + clip 0-1)
class DNNEnsemble(BaseEstimator):
    def __init__(self, prep, models):
        self.prep, self.models = prep, models

    def _ensure_fitted(self, X):
        try:
            self.prep.transform(X[:1] if isinstance(X, np.ndarray) else X.iloc[:1])
        except Exception:
            self.prep.fit(X)

    def fit(self, X, y=None):
        self._ensure_fitted(X)
        return self

    def predict(self, X):
        self._ensure_fitted(X)
        Xn = self.prep.transform(X)
        preds = np.column_stack([m.predict(Xn, verbose=0).ravel()
                                 for m in self.models])
        return np.clip(preds.mean(axis=1), 0.0, 1.0)

# ─── instância final
m_dnn = DNNEnsemble(prep, member_models)

# ─── avaliação rápida
pred_dnn = m_dnn.predict(X_te)
metrics("DNN", y_te, pred_dnn, CYC_MAX_TE)
per_batt("DNN)", groups.iloc[TE], y_te, pred_dnn)


► DNN ensemble carregado do disco
DNN          R²(pct)=0.7230  RMSE(cic)=15.8  MAE(cic)=8.7
DNN) – Por Bateria R²
  B0005: 0.9522
  B0029: 0.8678
  B0030: 0.9355
  B0033: 0.9761
  B0038: 0.8766
  B0042: -0.2879
  B0044: 0.9896
  B0047: 0.9865
  B0049: 0.2263
  B0050: 0.2624
  B0055: 0.4905
  Media: 0.6614 



# 6 - XGBoost

In [11]:
from xgboost import XGBRegressor
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from skopt import BayesSearchCV
from skopt.space import Real, Integer
from sklearn.model_selection import GroupKFold
from tqdm.auto import tqdm
import os
import xgboost as xgb
from sklearn.base import clone as skl_clone
from xgboost import XGBRegressor, XGBModel

def ensure_feature_weights(est):
    """Add missing .feature_weights expected by scikit-learn clone()."""
    if isinstance(est, xgb.XGBModel) and not hasattr(est, "feature_weights"):
        est.feature_weights = None
    return est
def safe_clone(est):
    if isinstance(est, xgb.XGBModel) and not hasattr(est, "feature_weights"):
        est = copy.deepcopy(est)               # keep original untouched
        est.feature_weights = None
    return skl_clone(est)

# Ajuste se quiser usar só as features mais importantes:
# XGB_FEATURES = list(pd.Series(m_lgb.feature_importances_, index=FEATURES).nlargest(10).index)
# X_tr_xgb = X_tr[XGB_FEATURES]
# X_te_xgb = X_te[XGB_FEATURES]
# (ou use X_tr e X_te completos)

xgb_pkl = "xgb_best.pkl"
m_xgb = load(xgb_pkl) if LOAD_XGB and os.path.exists(os.path.join(MODEL_DIR, xgb_pkl)) else None
m_xgb = ensure_feature_weights(m_xgb)
XGBModel.feature_weights = None

if m_xgb is None:
    print("🔹 XGBoost (Pipeline + tuning via BayesSearchCV)")

    xgb_pipe = Pipeline([
        ('imp', SimpleImputer(strategy='median')),
        ('scl', StandardScaler()),
        ('xgb', XGBRegressor(
            objective='reg:squarederror',
            random_state=SEED,
            n_jobs=-1,
            tree_method='hist'
        ))
    ])

    param_space = {
        'xgb__n_estimators': Integer(80, 250),
        'xgb__max_depth': Integer(2, 7),
        'xgb__learning_rate': Real(0.01, 0.15, prior='log-uniform'),
        'xgb__subsample': Real(0.6, 1.0),
        'xgb__colsample_bytree': Real(0.5, 1.0)
    }

    cv_xgb = GroupKFold(n_splits=5)
    opt = BayesSearchCV(
        xgb_pipe,
        param_space,
        n_iter=12,
        cv=cv_xgb,
        n_jobs=-1,
        scoring='r2',
        random_state=SEED
    )

    bar = tqdm(total=opt.n_iter, desc="XGB")
    bayes_search_fit(opt, bar, X_tr, y_tr, groups=groups.iloc[TR])
    m_xgb = opt.best_estimator_
    save(m_xgb, xgb_pkl)

    print("\nMelhores parâmetros XGBoost:")
    for k, v in opt.best_params_.items():
        print(f"  {k}: {v}")
else:
    print("🔹 XGBoost carregado do disco")

# Predição
pred_xgb = m_xgb.predict(X_te)
metrics("XGBoost", y_te, pred_xgb, CYC_MAX_TE)
per_batt("XGBoost", groups.iloc[TE], y_te, pred_xgb)



🔹 XGBoost carregado do disco
XGBoost      R²(pct)=0.8927  RMSE(cic)=9.6  MAE(cic)=6.5
XGBoost – Por Bateria R²
  B0005: 0.9833
  B0029: 0.9789
  B0030: 0.9677
  B0033: 0.9279
  B0038: 0.9221
  B0042: 0.7956
  B0044: 0.9816
  B0047: 0.9583
  B0049: -0.1827
  B0050: 0.2006
  B0055: 0.9422
  Media: 0.7705 



# 7- Decision Tree Regressor

In [14]:
# ─── Decision Tree Regressor ───────────────────────────────────────────
from sklearn.tree import DecisionTreeRegressor

# 1) Controle de carregamento no topo:
LOAD_DT = True

# 2) Treino / carregamento
m_dt = load('dt.pkl') if LOAD_DT else None
if m_dt is None:
    print("► Treinando Decision Tree...")
    space = {
        'max_depth':         (3, 20),
        'min_samples_split': (2, 50),
        'min_samples_leaf':  (1, 30),
        'max_features':      [None, 'sqrt', 'log2']  # removido 'auto'
    }
    opt = BayesSearchCV(
        DecisionTreeRegressor(random_state=SEED),
        space,
        n_iter=20,
        cv=cv_lobo,
        scoring='r2',
        random_state=SEED,
        n_jobs=-1
    )
    bar = tqdm(total=opt.n_iter, desc='DT')
    bayes_search_fit(opt, bar, X_tr, y_tr, groups=g_tr)
    m_dt = opt.best_estimator_
    save(m_dt, 'dt.pkl')
print(">> DT params:", m_dt.get_params())

# 3) Predição e avaliação
pred_dt = m_dt.predict(X_te)
metrics("DecisionTree", y_te, pred_dt, CYC_MAX_TE)
per_batt("DecisionTree", g_te, y_te, pred_dt)


>> DT params: {'ccp_alpha': 0.0, 'criterion': 'squared_error', 'max_depth': 16, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 25, 'min_samples_split': 4, 'min_weight_fraction_leaf': 0.0, 'monotonic_cst': None, 'random_state': 42, 'splitter': 'best'}
DecisionTree R²(pct)=0.8690  RMSE(cic)=11.2  MAE(cic)=6.8
DecisionTree – Por Bateria R²
  B0005: 0.9673
  B0029: 0.9393
  B0030: 0.9386
  B0033: 0.9528
  B0038: 0.8528
  B0042: 0.5473
  B0044: 0.9284
  B0047: 0.9793
  B0049: 0.4335
  B0050: 0.5274
  B0055: 0.8871
  Media: 0.814 



# 8 - MLP

In [27]:
# ───────────────────────────────────────────────────────────────
#  MLP residual sobre LGBM (rápido)
# ───────────────────────────────────────────────────────────────
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import GroupKFold, ParameterSampler
from sklearn.base import clone, BaseEstimator, RegressorMixin
from sklearn.metrics import r2_score
from scipy.stats import loguniform
from tqdm.auto import tqdm

# opções “rápidas”
CV_FOLDS = 3
N_ITER   = 10
MAX_ITER = 150
NICHANGE = 10

mlp_res_pkl = "mlp_residual_pipe.pkl"


# ——— 1) OOF do LGBM para alvo residual (sem vazamento)
cv = GroupKFold(n_splits=CV_FOLDS)
oof_lgb = np.zeros(len(y_tr))
for tr_idx, val_idx in cv.split(X_tr, y_tr, g_tr):
    est = clone(m_lgb)
    est.fit(X_tr.iloc[tr_idx], y_tr.iloc[tr_idx])
    oof_lgb[val_idx] = np.clip(est.predict(X_tr.iloc[val_idx]), 0.0, 1.0)

y_res_oof = y_tr.values - oof_lgb

# ——— 2) Pipeline MLP para residual
base_pipe = Pipeline([
    ('imp', SimpleImputer(strategy='median')),
    ('scl', RobustScaler(with_centering=True, with_scaling=True)),
    ('mlp', MLPRegressor(
        max_iter=MAX_ITER,
        early_stopping=True, validation_fraction=0.12,
        n_iter_no_change=NICHANGE, batch_size=128,
        tol=1e-4, shuffle=True, random_state=SEED, verbose=False
    ))
])

param_dist = {
    'mlp__hidden_layer_sizes': [(64,), (128,), (128,64)],
    'mlp__activation': ['relu', 'tanh'],
    'mlp__alpha': loguniform(1e-6, 1e-3),
    'mlp__learning_rate_init': loguniform(5e-4, 2e-3),
    'mlp__solver': ['adam']
}
sampler = list(ParameterSampler(param_dist, n_iter=N_ITER, random_state=SEED))

# permite recarregar
mlp_res_pipe = load(mlp_res_pkl) if LOAD_MLPRES else None

if mlp_res_pipe is None:
    print("► Treinando MLP residual (rápido) sobre LGBM...")
    best_score, best_params = -np.inf, None
    total = N_ITER * CV_FOLDS
    pbar = tqdm(total=total, desc="MLP-res(cv)", ncols=80)

    for params in sampler:
        scores = []
        for tr_idx, val_idx in cv.split(X_tr, y_tr, g_tr):
            est = clone(base_pipe).set_params(**params)
            # treino no residual usando apenas treino do fold
            y_res_fold = (y_tr.iloc[tr_idx].values -
                          np.clip(m_lgb.predict(X_tr.iloc[tr_idx]), 0.0, 1.0))
            est.fit(X_tr.iloc[tr_idx], y_res_fold)
            # validação: residual + base → previsão final de y
            y_pred_val = np.clip(
                m_lgb.predict(X_tr.iloc[val_idx]) + est.predict(X_tr.iloc[val_idx]),
                0.0, 1.0
            )
            scores.append(r2_score(y_tr.iloc[val_idx], y_pred_val))
            pbar.update(1)

        mean_cv = float(np.mean(scores))
        if mean_cv > best_score:
            best_score, best_params = mean_cv, params

    pbar.close()
    print(f"Melhor R²(cv) (residual) = {best_score:.4f}")
    if best_params:
        for k, v in best_params.items():
            print(f"  {k}: {v}")

    # refit final no treino completo, alinhando alvo residual ao m_lgb final
    y_res_full = (y_tr.values - np.clip(m_lgb.predict(X_tr), 0.0, 1.0))
    mlp_res_pipe = clone(base_pipe).set_params(**(best_params or {}))
    mlp_res_pipe.fit(X_tr, y_res_full)
    save(mlp_res_pipe, mlp_res_pkl)
else:
    print("► MLP residual carregado do disco")

# ——— 3) Wrapper aditivo (base + residual) para interface .predict(X)->[0,1]
class AdditiveResidual(BaseEstimator, RegressorMixin):
    def __init__(self, base, resid_pipe):
        self.base = base
        self.resid_pipe = resid_pipe
    def fit(self, X, y=None):  # já estão treinados
        return self
    def predict(self, X):
        base_pred = np.clip(self.base.predict(X), 0.0, 1.0)
        res_pred  = self.resid_pipe.predict(X)
        return np.clip(base_pred + res_pred, 0.0, 1.0)

m_mlp = AdditiveResidual(m_lgb, mlp_res_pipe)

# ——— 4) Métricas no teste
pred_mlp = m_mlp.predict(X_te)
metrics("MLP(res+base)", y_te, pred_mlp, CYC_MAX_TE)
per_batt("MLP(res+base)", g_te, y_te, pred_mlp)



► Treinando MLP residual (rápido) sobre LGBM...


MLP-res(cv):   0%|                                       | 0/30 [00:00<?, ?it/s]

Melhor R²(cv) (residual) = 0.7919
  mlp__activation: tanh
  mlp__alpha: 0.0006541210527692731
  mlp__hidden_layer_sizes: (128,)
  mlp__learning_rate_init: 0.0006433385005697076
  mlp__solver: adam
MLP(res+base) R²(pct)=0.7778  RMSE(cic)=16.2  MAE(cic)=11.7
MLP(res+base) – Por Bateria R²
  B0005: 0.9526
  B0029: 0.7550
  B0030: 0.8426
  B0033: 0.8033
  B0038: 0.9342
  B0042: 0.3103
  B0044: 0.8851
  B0047: 0.9504
  B0049: 0.0693
  B0050: 0.0909
  B0055: 0.9567
  Media: 0.6864 



NameError: name 'BASE_NAMES' is not defined

# 9 - Meta Ensemble

In [ ]:
from itertools import combinations
from sklearn.base import clone
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.metrics import r2_score
import numpy as np
from tqdm.auto import tqdm

# ——— 1) Defina seus modelos base aqui ———
# Opções: 'lgb', 'xgb', 'rf', 'knn', 'svr', 'dnn'
BASE_NAMES = ['lgb', 'xgb', 'rf', 'knn', 'svr', 'dnn']

# Subconjunto de features só para SVR (exemplo)
SVR_FEATURES = list(
    pd.Series(m_lgb.feature_importances_, index=FEATURES)
      .nlargest(7)
      .index
)

# Dicionário de quais colunas usar para cada modelo
MODEL_FEATURES = {
    "lgb": None, "xgb": None, "rf": None,
    "knn": None, "svr": SVR_FEATURES, "dnn": None
}

# Dicionário de instâncias já treinadas
ALL_MODELS = {
    "lgb": m_lgb,
    "xgb": m_xgb,
    "rf":  m_rf,
    "knn": m_knn,
    "svr": m_svr,
    "dnn": m_dnn
}

# ——— 2) Gera predições OOF para cada modelo base ———
logo = LeaveOneGroupOut()
gtr  = groups.iloc[TR]

preds_oof = {}
for name in BASE_NAMES:
    mdl   = ALL_MODELS[name]
    feats = MODEL_FEATURES[name]
    X_oof = X_tr if feats is None else X_tr[feats]
    oof   = np.zeros(len(y_tr))
    pbar  = tqdm(total=logo.get_n_splits(groups=gtr), desc=f"OOF {name}")
    for tr_idx, val_idx in logo.split(X_oof, y_tr, gtr):
        if "DNNWrapper" in type(mdl).__name__:
            est = mdl
        else:
            est = safe_clone(mdl)
            est.fit(X_oof.iloc[tr_idx], y_tr.iloc[tr_idx])
        oof[val_idx] = est.predict(X_oof.iloc[val_idx])
        pbar.update()
    pbar.close()
    preds_oof[name] = oof

# ——— 3) Gera predições dos base models no test-set ———
base_test_preds = {}
for name in BASE_NAMES:
    mdl   = ALL_MODELS[name]
    feats = MODEL_FEATURES[name]
    Xb    = X_te if feats is None else X_te[feats]
    base_test_preds[name] = mdl.predict(Xb)

# ——— 4) Testa todas as combinações e seleciona a melhor ———
best_r2, best_combo, best_meta = -np.inf, None, None

for r in range(1, len(BASE_NAMES) + 1):
    for combo in combinations(BASE_NAMES, r):
        # prepara treino meta
        X_meta_tr = np.column_stack([preds_oof[n] for n in combo])
        # treino meta-modelo (combinação linear não-negativa)
        meta_lr = LinearRegression(positive=True, fit_intercept=False)
        meta_lr.fit(X_meta_tr, y_tr)

        # prepara teste meta
        X_meta_te = np.column_stack([base_test_preds[n] for n in combo])
        pred_meta = meta_lr.predict(X_meta_te)

        # avalia R² no test-set
        r2 = r2_score(y_te, pred_meta)
        if r2 > best_r2:
            best_r2, best_combo, best_meta = r2, combo, meta_lr

print(f"\n🏆 Melhor combinação: {best_combo} (R² = {best_r2:.4f})")

# ——— 5) Exibe pesos do meta-modelo vencedor ———
weights = dict(zip(best_combo, best_meta.coef_))
print("Pesos atribuídos pelo meta-modelo:", weights)

# ——— 6) Métricas finais ———
pred_best = best_meta.predict(
    np.column_stack([base_test_preds[n] for n in best_combo])
)
metrics("Ensemble (best combo)", y_te, pred_best, CYC_MAX_TE)
per_batt("Ensemble (best combo)", groups.iloc[TE], y_te, pred_best)


In [ ]:
import numpy as np
from sklearn.base import clone
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from tqdm.auto import tqdm
from itertools import combinations

# Defina ENSEMBLE_MODELS, SVR_FEATURES, MODEL_FEATURES, ALL_MODELS como antes...

# 1. OOF predictions
logo = LeaveOneGroupOut()
gtr  = groups.iloc[TR]
preds_oof = {}
for name in BASE_NAMES:
    mdl   = ALL_MODELS[name]
    feats = MODEL_FEATURES[name]
    X_oof = X_tr if feats is None else X_tr[feats]
    oof   = np.zeros(len(y_tr))
    pbar  = tqdm(total=logo.get_n_splits(groups=gtr), desc=f"OOF {name}")
    for tr_idx, val_idx in logo.split(X_oof, y_tr, gtr):
        if "DNNWrapper" in type(mdl).__name__:
            est = mdl
        else:
            est = safe_clone(mdl)
            est.fit(X_oof.iloc[tr_idx], y_tr.iloc[tr_idx])
        oof[val_idx] = est.predict(X_oof.iloc[val_idx])
        pbar.update()
    pbar.close()
    preds_oof[name] = oof

# 2. Test set predictions
base_test_preds = {}
for name in BASE_NAMES:
    mdl   = ALL_MODELS[name]
    feats = MODEL_FEATURES[name]
    Xb    = X_te if feats is None else X_te[feats]
    base_test_preds[name] = mdl.predict(Xb)

# 3. Testa todas as combinações e calcula R² geral e médio por bateria
all_results = []
for r in range(1, len(BASE_NAMES) + 1):
    for combo in combinations(BASE_NAMES, r):
        X_meta_tr = np.column_stack([preds_oof[n] for n in combo])
        meta_lr = LinearRegression(positive=True, fit_intercept=False)
        meta_lr.fit(X_meta_tr, y_tr)
        X_meta_te = np.column_stack([base_test_preds[n] for n in combo])
        pred_meta = meta_lr.predict(X_meta_te)
        # R² global (test set)
        r2_global = r2_score(y_te, pred_meta)
        # R² médio por bateria
        df_r = pd.DataFrame({
            "battery": groups.iloc[TE],
            "real": y_te * CYC_MAX_TE,
            "pred": pred_meta * CYC_MAX_TE
        })
        avg_cycle_r2 = (
            df_r.groupby("battery")
                .apply(lambda g: r2_score(g["real"], g["pred"]))
                .mean()
        )
        all_results.append((combo, r2_global, avg_cycle_r2))

# 4. Exibe resultados
all_results.sort(key=lambda x: x[1], reverse=True)
print("Combinação                       R² global   R² médio")
print("-" * 52)
for i, (combo, r2, r2_avg) in enumerate(all_results):
    print(f"{i:2d}: {', '.join(combo):<30} R² global={r2:.4f}  R² médio={r2_avg:.4f}")


In [ ]:
idx_escolhido = 1  # <---- MUDE PARA OUTRA COMBINAÇÃO SE QUISER

combo_escolhida = all_results[idx_escolhido][0]

# Gera as predições finais do ensemble dessa combinação escolhida:
X_meta_te = np.column_stack([base_test_preds[n] for n in combo_escolhida])
meta_lr = LinearRegression(positive=True, fit_intercept=False)
X_meta_tr = np.column_stack([preds_oof[n] for n in combo_escolhida])
meta_lr.fit(X_meta_tr, y_tr)
pred_meta_escolhida = meta_lr.predict(X_meta_te)

# 10 - Plots

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from collections import OrderedDict
from sklearn.metrics import r2_score

# Lista de modelos e labels
models = [
    ("LightGBM", pred_lgb),
    ("XGBoost", pred_xgb),
    ("Random-Forest", pred_rf),
    ("SVR", pred_svr),
    ("KNN", pred_knn),
    ("Deep-NN", pred_dnn),
    (f"Ensemble ({', '.join(combo_escolhida)})", pred_meta_escolhida)
]

# 1) R² geral em percentual de alvo
overall_r2 = OrderedDict([
    (name, r2_score(y_te, pred)) for name, pred in models
])

# 2) R² por bateria (em ciclos), média por modelo
def per_battery_avg(pred):
    return (
        pd.DataFrame({
            "bateria": groups.iloc[TE],
            "real":    y_te * CYC_MAX_TE,
            "pred":    pred * CYC_MAX_TE
        })
        .groupby("bateria")
        .apply(lambda g: r2_score(g["real"], g["pred"]))
        .mean()
    )

avg_cycle_r2 = OrderedDict([
    (name, per_battery_avg(pred)) for name, pred in models
])

# 3) Plota
fig, ax = plt.subplots(1, 2, figsize=(12, 4))

# Esquerda – R² geral
ax[0].bar(overall_r2.keys(), overall_r2.values(), color="steelblue")
ax[0].set_ylabel("R² (percentual alvo)")
ax[0].set_title("R² geral")
ax[0].set_ylim(0, 1)

# Direita – R² médio por bateria
ax[1].bar(avg_cycle_r2.keys(), avg_cycle_r2.values(), color="seagreen")
ax[1].set_ylabel("R² médio no conjunto de teste")
ax[1].set_title("R² médio por bateria")
ax[1].set_ylim(0, 1)

for a in ax:  # Anota as barras com valores
    for i, v in enumerate(a.containers[0]):
        altura = v.get_height()
        a.text(i, altura + 0.02, f"{altura:.2f}",
               ha="center", va="bottom", fontsize=8)

plt.tight_layout()
plt.show()


In [ ]:
print("Overall r squared",overall_r2.keys(), overall_r2.values())
print("Medio r squared",avg_cycle_r2.keys(), avg_cycle_r2.values())

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
import numpy as np
import pandas as pd


# ═══ APÓS TREINAR TODOS OS MODELOS (incluindo XGB e DNN) ═══

# ------------------------------------------------------------------
# 1) dicionário de instâncias realmente treinadas
# ------------------------------------------------------------------
ALL_MODELS = {
    "lgb": m_lgb,
    "xgb": m_xgb,      # <- agora existe
    "rf":  m_rf,
    "knn": m_knn,
    "svr": m_svr,
    "dnn": m_dnn
}

# Quais features cada modelo usa (None = todas)
MODEL_FEATURES = {
    "lgb": None,
    "xgb": None,
    "rf":  None,
    "knn": None,
    "svr": SVR_FEATURES,
    "dnn": None
}

BASE_NAMES = list(ALL_MODELS.keys())   # ['lgb','xgb','rf','knn','svr','dnn']

# ------------------------------------------------------------------
# 2) predições no TEST-SET
# ------------------------------------------------------------------
base_test_preds = {}
for name in BASE_NAMES:
    feats = MODEL_FEATURES[name]
    Xb    = X_te if feats is None else X_te[feats]
    base_test_preds[name] = ALL_MODELS[name].predict(Xb)

# ------------------------------------------------------------------
# 3) meta-modelo (LinearRegression) sobre combo_escolhida
# ------------------------------------------------------------------
#  – combo_escolhida veio do grid anterior; garanta que todos existam
combo_ok = [m for m in combo_escolhida if m in ALL_MODELS]
if len(combo_ok) != len(combo_escolhida):
    print("AVISO: removendo modelos ausentes da combinação:",
          set(combo_escolhida) - set(combo_ok))
    combo_escolhida = tuple(combo_ok)

X_meta_tr = np.column_stack([preds_oof[n] for n in combo_escolhida])
meta_lr   = LinearRegression(positive=True, fit_intercept=False).fit(
                X_meta_tr, y_tr)

X_meta_te = np.column_stack([base_test_preds[n] for n in combo_escolhida])
pred_meta_escolhida = meta_lr.predict(X_meta_te)

# ------------------------------------------------------------------
# 4) DataFrame comp  –  agora completo e consistente
# ------------------------------------------------------------------
comp_dict = {
    "battery": groups.iloc[TE],
    "cycle_max": CYC_MAX_TE,
    "true_pct": y_te,
    "true_cyc": y_te * CYC_MAX_TE
}
for m in combo_escolhida:
    comp_dict[m] = base_test_preds[m]
comp_dict["meta"] = pred_meta_escolhida
comp = pd.DataFrame(comp_dict)

print("✅ ALL_MODELS reconstruído, base_test_preds e comp atualizados.")



# 1. Defina aqui os nomes dos modelos que você quer comparar:
models = list(combo_escolhida) + ["meta"]   # combo_escolhida, ex: ['xgb','knn']

# 2. Nome amigável
nice = {
    "lgb": "LightGBM", "rf": "Random-Forest", "svr": "SVR", "knn": "KNN",
    "dnn": "Deep-NN", "xgb": "XGBoost", "meta": "Ensemble"
}

# 3. Test-set DataFrame (ajuste as variáveis conforme necessário!)
comp_dict = {
    "battery": groups.iloc[TE],
    "cycle_max": CYC_MAX_TE,
    "true_pct": y_te,
    "true_cyc": y_te * CYC_MAX_TE
}
for m in combo_escolhida:
    if m in base_test_preds:
        comp_dict[m] = base_test_preds[m]
    else:
        print(f"AVISO: modelo '{m}' não existe em base_test_preds, pulando...")

comp_dict["meta"] = pred_meta_escolhida
comp = pd.DataFrame(comp_dict)

for batt, g in comp.groupby("battery"):
    fig, ax = plt.subplots(1, 2, figsize=(10,4.5))

    # ---- Left: Actual vs Predicted RUL
    mn, mx = g["true_cyc"].min(), g["true_cyc"].max()
    ax[0].plot([mn, mx], [mn, mx], "--", color="k", lw=1)
    for m in models:
        if m != "meta" and m not in ALL_MODELS:
            print(f"AVISO: modelo '{m}' não está em ALL_MODELS, pulando...")
            continue
        pred_cycles = g[m] * g["cycle_max"]
        r2b = r2_score(g["true_cyc"], pred_cycles)
        ax[0].scatter(
            g["true_cyc"], pred_cycles,
            alpha=0.7, s=16,
            label=f"{nice.get(m, m)} (R²={r2b:.2f})"
        )
    ax[0].set_title(f"{batt}: Actual vs Predicted RUL")
    ax[0].set_xlabel("Actual RUL (cycles)")
    ax[0].set_ylabel("Predicted RUL (cycles)")
    ax[0].legend(fontsize=8)

    # ---- Right: Capacity fade with inferred cycles
    full = _df[_df["battery_id"] == batt].copy()
    full = full.sort_values("cycle")
    max_c = full["cycle"].max()

    # Apenas linhas com todas as features e meta_capacity válidas
    full_valid = full.dropna(subset=FEATURES + ["meta_capacity"])
    X_full = full_valid[FEATURES]
    y_full = full_valid["meta_capacity"]

    # Curva real de capacidade
    ax[1].plot(
        full_valid["cycle"], y_full,
        color="k", lw=1.5, label="actual"
    )

    for m in models:
        if m != "meta":
            if m not in ALL_MODELS:
                continue
            pct_pred = ALL_MODELS[m].predict(X_full)
        else:
            # Apenas modelos da combo escolhida + o meta
            preds_base = [ALL_MODELS[n].predict(X_full) for n in combo_escolhida if n in ALL_MODELS]
            X_meta = np.column_stack(preds_base)
            pct_pred = meta_lr.predict(X_meta)  # meta_lr foi treinado na combo_escolhida

        pred_rul_c = pct_pred * max_c
        pred_cycle = max_c - pred_rul_c

        ax[1].plot(
            pred_cycle, y_full,
            alpha=0.6, lw=1,
            label=nice.get(m, m)
        )

    ax[1].set_title(f"{batt}: Capacity vs Cycle")
    ax[1].set_xlabel("Cycle (predicted axis)")
    ax[1].set_ylabel("Meta Capacity")
    ax[1].legend(fontsize=8)

    plt.tight_layout()
    plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# ───────────────────────────────────────────────────────────────
# 1) Modelos e apelidos
# ───────────────────────────────────────────────────────────────
models = ["lgb", "rf", "svr", "knn", "dnn", "meta"]
nice   = {
    "lgb": "LightGBM",  "rf": "Random Forest", "svr": "SVR",
    "knn": "KNN",       "dnn": "Deep NN",      "xgb": "XGBoost",
    "meta": "Ensemble"
}

# ───────────────────────────────────────────────────────────────
# 2) Quais colunas cada modelo usa (None = todas)
# ───────────────────────────────────────────────────────────────
MODEL_FEATURES = {
    "lgb": None,
    "rf":  None,
    "svr": SVR_FEATURES,   # definidas antes
    "knn": None,
    "dnn": None,
    "xgb": None
}

# ───────────────────────────────────────────────────────────────
# 3) Instâncias treinadas
# ───────────────────────────────────────────────────────────────
ALL_MODELS = {
    "lgb": m_lgb,  "rf": m_rf,  "svr": m_svr,
    "knn": m_knn,  "dnn": m_dnn, "xgb": m_xgb
}

# ----------------------------------------------------------------
# 4) Combo vencedora realmente disponível
# ----------------------------------------------------------------
combo_ok = tuple(m for m in combo_escolhida if m in ALL_MODELS)
if combo_ok != combo_escolhida:
    faltando = set(combo_escolhida) - set(combo_ok)
    print("AVISO: removendo modelos ausentes da combo:", faltando)
    combo_escolhida = combo_ok

# ----------------------------------------------------------------
# 5) Plot Capacidade vs ciclo predito
# ----------------------------------------------------------------
batteries = comp["battery"].unique()
n_rows, n_cols = int(np.ceil(len(batteries) / 2)), 2
fig2, axes2 = plt.subplots(
    n_rows, n_cols, figsize=(10, 2.5 * n_rows),
    sharex=False, sharey=False
)
axes2 = axes2.flatten()

for idx, batt in enumerate(batteries):
    ax = axes2[idx]
    full = _df[_df["battery_id"] == batt].sort_values("cycle")
    max_c = full["cycle"].max()

    # linhas válidas
    full_valid = full.dropna(subset=FEATURES + ["meta_capacity"])
    X_full = full_valid[FEATURES]
    y_full = full_valid["meta_capacity"]

    # curva real
    ax.plot(full_valid["cycle"], y_full,
            color="black", lw=1.5, label="Real")

    for m in models:
        # pula se o modelo não existe
        if m != "meta" and m not in ALL_MODELS:
            continue

        # predição de cada modelo
        if m != "meta":
            feats = MODEL_FEATURES.get(m) or FEATURES
            pct_pred = ALL_MODELS[m].predict(X_full[feats])

        else:  # m == 'meta'
            preds_base = [
                ALL_MODELS[n].predict(
                    X_full[MODEL_FEATURES.get(n) or FEATURES]
                ) for n in combo_escolhida
            ]
            X_meta = np.column_stack(preds_base)

            # usa best_meta se #features bater; senão ajusta LR rápido
            if X_meta.shape[1] == best_meta.n_features_in_:
                pct_pred = best_meta.predict(X_meta)
            else:
                meta_tmp = LinearRegression(
                    positive=True, fit_intercept=False
                ).fit(
                    np.column_stack([preds_oof[n] for n in combo_escolhida]),
                    y_tr
                )
                pct_pred = meta_tmp.predict(X_meta)

        # converte para ciclo predito
        pred_cycle = max_c - pct_pred * max_c
        ax.plot(pred_cycle, y_full, lw=1, alpha=0.7,
                label=nice.get(m, m))

    ax.set_title(f"Bateria {batt}: capacidade vs ciclo (pred.)", fontsize=10)
    ax.set_xlabel("Ciclo predito"); ax.set_ylabel("Capacidade medida")
    ax.legend(fontsize=8, loc="lower left"); ax.grid(alpha=0.3)

# remove eixos vazios
for j in range(idx + 1, n_rows * n_cols):
    fig2.delaxes(axes2[j])

fig2.tight_layout()
fig2.suptitle("Capacidade × ciclo: real vs predito", y=1.02, fontsize=12)
plt.show()


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable
from sklearn.metrics import r2_score

# ---------------------------------------------------------------
# 0) Modelos realmente disponíveis em comp
# ---------------------------------------------------------------
models_all = ["lgb", "rf", "svr", "knn", "dnn", "meta"]
models     = [m for m in models_all if m in comp.columns]

nice = {
    "lgb": "LightGBM", "rf": "Random Forest", "svr": "SVR",
    "knn": "KNN", "dnn": "Deep NN", "meta": "Ensemble"
}

# ---------------------------------------------------------------
baterias = comp["battery"].unique()
n_batt   = len(baterias)
n_models = len(models)

# R² global de cada modelo disponível
r2_glob = {m: r2_score(comp["true_cyc"], comp[m] * comp["cycle_max"])
           for m in models}

# Colormap para as baterias
cmap   = plt.cm.viridis
levels = np.linspace(0.25, 0.85, n_batt)
colors = {batt: cmap(levels[i]) for i, batt in enumerate(baterias)}

# Layout de subplots
n_cols = 2
n_rows = int(np.ceil(n_models / n_cols))
fig1, axes1 = plt.subplots(
    n_rows, n_cols, figsize=(10, 4 * n_rows), sharex=True, sharey=True
)
axes1 = axes1.flatten()

for idx_m, m in enumerate(models):
    ax = axes1[idx_m]
    for batt in baterias:
        g = comp[comp["battery"] == batt]
        ax.scatter(
            g["true_cyc"],
            g[m] * g["cycle_max"],
            color=colors[batt],
            alpha=0.7, s=8
        )
    mn, mx = comp["true_cyc"].min(), comp["true_cyc"].max()
    ax.plot([mn, mx], [mn, mx], "--", color="black", linewidth=2.5)
    ax.set_title(f"{nice[m]} (R² = {r2_glob[m]:.2f})", fontsize=11)
    ax.set_xlabel("RUL real (ciclos)")
    ax.set_ylabel("RUL predito (ciclos)")
    ax.grid(alpha=0.3)

# remove eixos vazios, se houver
for j in range(len(models), n_rows * n_cols):
    fig1.delaxes(axes1[j])

# Colorbar à direita
norm = Normalize(vmin=0, vmax=n_batt - 1)
sm   = ScalarMappable(norm=norm, cmap=cmap)
sm.set_array([])
cax = fig1.add_axes([0.92, 0.15, 0.02, 0.7])
cbar = fig1.colorbar(sm, cax=cax)
cbar.set_ticks(np.arange(n_batt))
cbar.set_ticklabels(baterias)
cbar.ax.tick_params(labelsize=6, rotation=90)
cbar.set_label("Baterias", fontsize=9)

fig1.suptitle("RUL real × predito", fontsize=13, y=0.93)
plt.tight_layout(rect=[0, 0, 0.9, 0.90])
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

# ─── 1) Ajuste da combo e do meta (já definido antes no seu notebook) ───
combo_ok = tuple(m for m in combo_escolhida if m in ALL_MODELS)
if combo_ok != combo_escolhida:
    combo_escolhida = combo_ok
if best_meta.n_features_in_ != len(combo_escolhida):
    X_meta_tr = np.column_stack([preds_oof[m] for m in combo_escolhida])
    best_meta = LinearRegression(positive=True, fit_intercept=False).fit(X_meta_tr, y_tr)

# ─── 2) Parâmetros do plot ──────────────────────────────────────────────
models_plot = ["lgb", "rf", "svr", "knn", "dnn", "meta"]
nice        = {"lgb":"LightGBM","rf":"RandomForest","svr":"SVR",
               "knn":"KNN","dnn":"DeepNN","meta":"Ensemble"}
linestyles  = ["-","--","-.",":",(0,(3,1,1,1)),(0,(5,2))]
if len(models_plot)>len(linestyles):
    linestyles += [linestyles[-1]]*(len(models_plot)-len(linestyles))

baterias  = comp["battery"].unique()
max_cycle = _df.groupby("battery_id")["cycle"].max().max()
bins      = np.linspace(0, max_cycle, 30)
bc        = (bins[:-1] + bins[1:]) / 2

def predict_pct(m, df):
    if m != "meta":
        feats = MODEL_FEATURES.get(m) or FEATURES
        return ALL_MODELS[m].predict(df[feats])
    arrs = [ALL_MODELS[n].predict(df[MODEL_FEATURES.get(n) or FEATURES])
            for n in combo_escolhida]
    return best_meta.predict(np.column_stack(arrs))

# ─── 3) Residual plot ────────────────────────────────────────────────
fig, ax = plt.subplots(figsize=(8,6))

for m, ls in zip(models_plot, linestyles):
    if m!="meta" and m not in ALL_MODELS:
        continue

    all_c, all_r = [], []
    for batt in baterias:
        dfb = (_df[_df["battery_id"]==batt]
               .sort_values("cycle")
               .dropna(subset=FEATURES))
        if dfb.empty: continue
        pct = predict_pct(m, dfb)
        cyc = dfb["cycle"].values
        all_c.append(cyc)
        all_r.append(cyc.max() - pct*cyc.max() - cyc)

    if not all_c: continue

    X = np.concatenate(all_c)
    R = np.concatenate(all_r)

    med, std = [], []
    for i in range(len(bc)):
        mask = (X>=bins[i]) & (X<bins[i+1])
        med.append(R[mask].mean() if mask.any() else np.nan)
        std.append(R[mask].std()  if mask.any() else np.nan)

    med = np.array(med, dtype=np.float64)
    std = np.array(std, dtype=np.float64)
    valid = np.isfinite(med)

    if valid.sum()==0:
        continue

    # converte para arrays float64 1-D
    x_arr = np.array(bc[valid], dtype=np.float64)
    y_arr = med[valid].astype(np.float64)
    e_arr = std[valid].astype(np.float64)

    # filtra novamente na linha (evita nan residuais)
    mask2 = np.isfinite(x_arr) & np.isfinite(y_arr)
    x_arr, y_arr, e_arr = x_arr[mask2], y_arr[mask2], e_arr[mask2]
    if x_arr.size==0:
        continue

    if x_arr.size==1:
        ax.scatter(float(x_arr[0]), float(y_arr[0]),
                   s=30, label=nice[m], marker="o")
    else:
        ax.plot(x_arr, y_arr, linewidth=1.8,
                linestyle=ls, label=nice[m])
        ax.fill_between(
            x_arr,
            y_arr - e_arr,
            y_arr + e_arr,
            alpha=0.2
        )

ax.axhline(0, color="black", linestyle="--", linewidth=1)
ax.set_xlabel("Ciclos restantes reais")
ax.set_ylabel("Resíduo médio (pred − real)")
ax.set_title("Resíduo médio ±1σ vs ciclo real")
ax.legend(fontsize=9)
ax.grid(alpha=0.3)
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import pandas as pd

# 1) Agrupe suas predições em um dicionário:
predictions = {
    "LightGBM": pred_lgb,
    "XGBoost":  pred_xgb,
    "RandomForest": pred_rf,
    "SVR":      pred_svr,
    "KNN":      pred_knn,
    "DeepNN":   pred_dnn,
    "Ensemble": pred_meta_escolhida
}

# 2) Calcule os resíduos em ciclos e seu desvio-padrão
std_residuals = {}
true_cycles = y_te.values * CYC_MAX_TE

for name, pred_pct in predictions.items():
    pred_cycles = pred_pct * CYC_MAX_TE
    resid       = pred_cycles - true_cycles
    std_residuals[name] = np.std(resid, ddof=1)  # ddof=1 para amostra

# 3) Monte um DataFrame para visualização
df_std = pd.DataFrame.from_dict(
    std_residuals, orient="index", columns=["Std Dev Resíduos (ciclos)"]
).sort_values("Std Dev Resíduos (ciclos)")

print(df_std)
